In [20]:
# example for frameworks
from pydasa.dimensional.fundamental import Dimension
from pydasa.dimensional.vaschy import Schema


# Define custom FDUs
custom_fdus = [
    {"_idx": 0, "_sym": "T", "_unit": "s", "_name": "Time"},
    {"_idx": 1, "_sym": "D", "_unit": "bit", "_name": "Data"},
    {"_idx": 2, "_sym": "O", "_unit": "op", "_name": "Operations"}
]

# Create schema with custom FDUs
schema = Schema(_fwk="CUSTOM", _fdu_lt=custom_fdus)

# OR use Dimension class to create the FDUs
custom_fdus = [
    Dimension(_idx=0, _sym="T", _unit="s", _name="Time"),
    Dimension(_idx=1, _sym="D", _unit="bit", _name="Data"),
    Dimension(_idx=2, _sym="O", _unit="op", _name="Operations")
]

schema = Schema(_fwk="CUSTOM", _fdu_lt=custom_fdus)


In [21]:
# example for variables
from pydasa import Variable, Schema

schema = Schema(_fwk="COMPUTATION")  # Time, Data, Operations
print(schema)

# INPUT: Problem size (minimal specification for dimensional model)
N = Variable(
    _name="Problem Size",
    _sym="N",
    _cat="IN",
    _dims="S",  # Data dimension
    _units="elements",
    _schema=schema
)

# INPUT with numerical bounds for sensitivity analysis
T = Variable(
    _name="Time Budget",
    _sym="T",
    _cat="IN",
    _dims="T",
    _units="seconds",
    _min=1.0,
    _max=10.0,
    _schema=schema
)

# INPUT with full distribution for Monte Carlo
throughput = Variable(
    _name="Throughput",
    _sym="R",
    _cat="IN",
    _dims="S*T^-1",
    _units="MB/s",
    _dist_type="normal",
    _dist_params={"mean": 100.0, "std": 10.0},
    _schema=schema
)

# OUTPUT: Depends on other variables (distribution computed automatically)
latency = Variable(
    _name="Latency",
    _sym="L",
    _cat="OUT",
    _dims="T",
    _units="milliseconds",
    _depends=["N", "R"],  # Function of problem size and throughput
    _schema=schema
)

Schema(sym='', fwk='COMPUTATION', alias='', idx=-1, name='', description='', fdu_lt=[Dimension(_sym='T', _fwk='COMPUTATION', _alias='', _idx=0, _name='Time', description='Duration of an event or interval.', _unit='s'), Dimension(_sym='S', _fwk='COMPUTATION', _alias='', _idx=1, _name='Space', description='Physical extent in three dimensions.', _unit='bit'), Dimension(_sym='N', _fwk='COMPUTATION', _alias='', _idx=2, _name='Complexity', description='Measure of interconnectedness or intricacy in a system.', _unit='op')], fdu_map={'T': Dimension(_sym='T', _fwk='COMPUTATION', _alias='', _idx=0, _name='Time', description='Duration of an event or interval.', _unit='s'), 'S': Dimension(_sym='S', _fwk='COMPUTATION', _alias='', _idx=1, _name='Space', description='Physical extent in three dimensions.', _unit='bit'), 'N': Dimension(_sym='N', _fwk='COMPUTATION', _alias='', _idx=2, _name='Complexity', description='Measure of interconnectedness or intricacy in a system.', _unit='op')}, fdu_symbols=['T

In [22]:
# Example of dimensional model/matrix
# Simple example: Free fall
from pydasa import Variable, Schema, Matrix

# Configure Schema
schema = Schema(_fwk="PHYSICAL")

# Define variables (only dimensional quantities)
variables = {
    "h": Variable(_name="Height", _sym="h", _cat="OUT", 
                    _dims="L", _units="m", relevant=True, _schema=schema),
    "v": Variable(_name="Velocity", _sym="v", _cat="IN",
                    _dims="L*T^-1", _units="m/s", relevant=True, _schema=schema),
    "g": Variable(_name="Gravity", _sym="g", _cat="CTRL",
                    _dims="L*T^-2", _units="m/s^2", relevant=True, _schema=schema)
}

# Create and solve dimensional model
model = Matrix(_name="Free Fall", _schema=schema, _variables=variables)
model.create_matrix()
model.solve_matrix()

# Display results
for sym, coef in model.coefficients.items():
    print(f"{sym}: {coef.pi_expr}")

\Pi_{0}: \frac{h*g}{v^{2}}


In [23]:
# coeffcicient example
import numpy as np
from pydasa import Variable, Schema, Matrix

# Schema and variables from Matrix example
schema = Schema(_fwk="PHYSICAL")
variables = {
    "R": Variable(_name="Range",
                    _sym="R", _cat="OUT",
                    _dims="L",
                    relevant=True,
                    _std_min=10, _std_max=100,
                    _units="m", _schema=schema),
    "v": Variable(_name="Velocity",
                    _sym="v", _cat="IN",
                    _dims="L*T^-1",
                    relevant=True,
                    _std_min=5, _std_max=50,
                    _units="m/s", _schema=schema),
    "g": Variable(_name="Gravity",
                    _sym="g", _cat="CTRL",
                    _dims="L*T^-2",
                    relevant=True,
                    _std_setpoint=9.81, _units="m/s^2",
                    _schema=schema)
}

# Generate dimensional matrix and coefficients
model = Matrix(_name="Projectile",
                _schema=schema,
                _variables=variables)
model.create_matrix()
model.solve_matrix()  # Produces π₁ = R·g/v², π₂ = θ (dimensionless)

# Access coefficient with inherited numerical properties
pi_0 = model.coefficients["\Pi_{0}"]             # Coefficient object for pi_0
print(f"Formula: {pi_0.pi_expr}")                # "R*g/v**2"
print(f"Variables: {pi_0.variables.keys()}")     # dict_keys(['R', 'g', 'v'])
print(f"Exponents: {pi_0.var_dims}")             # {'R': 1, 'g': 1, 'v': -2}
    
# can setup bounds and discretization to compute data
pi_0.min = variables["R"].std_min * variables["g"].std_setpoint / (variables["v"].std_max ** 2)  # (10*9.81)/(50**2) = 0.039
pi_0.max = variables["R"].std_max * variables["g"].std_setpoint / (variables["v"].std_min ** 2)  # (100*9.81)/(5**2) = 39.24
pi_0.step = 0.1  # Custom step for analysis
data = np.arange(pi_0.min, pi_0.max, pi_0.step)  # Grid data for analysis

pi_0.data = np.array(data)
pi_0.mean = np.mean(pi_0._data)    # Average dimensionless coefficient
pi_0.dev = np.std(pi_0._data)      # Coefficient uncertainty
print(f"Monte Carlo mean: {pi_0._mean}, std: {pi_0._dev}")

# or generate data with Monte Carlo sampling using variable bounds
# Monte Carlo simulation using inherited statistics
sample = []
for _ in range(100):
    R_sample = np.random.uniform(variables["R"].std_min, variables["R"].std_max)
    v_sample = np.random.uniform(variables["v"].std_min, variables["v"].std_max)
    
    # Compute pi_0 for each sample
    point = pi_0.calculate_setpoint(dict(R=R_sample,
                                         v=v_sample,
                                         g=variables["g"].std_setpoint))
    # Compute pi_0 for each sample
    sample.append(point)

pi_0.data = np.array(sample)
pi_0.mean = np.mean(pi_0.data)    # Average dimensionless coefficient
pi_0.dev = np.std(pi_0.data)      # Coefficient uncertainty
print(f"Monte Carlo mean: {pi_0.mean}, std: {pi_0.dev}")

Formula: \frac{R*g}{v^{2}}
Variables: dict_keys(['v', 'R', 'g'])
Exponents: {'v': -2, 'R': 1, 'g': 1}
Monte Carlo mean: 19.63924, std: 11.344896062400338
Monte Carlo mean: 2.2545587279534693, std: 4.604567688047786


In [24]:
# Dimensional Engine Example
from pydasa import Variable, Schema, AnalysisEngine
from pydasa.dimensional.fundamental import Dimension
import json
import os

# Define custom framework (T, M, L only - typical for fluid mechanics)
custom_fdus = [
    Dimension(_idx=0, _sym="T", _unit="s", _name="Time"),
    Dimension(_idx=1, _sym="M", _unit="kg", _name="Mass"),
    Dimension(_idx=2, _sym="L", _unit="m", _name="Length")
]
schema = Schema(_fwk="CUSTOM", _fdu_lt=custom_fdus)

# Define variables (only 4 relevant for Reynolds number)
variables = {
    "\\rho": Variable(_name="Density", _sym="\\rho", _cat="IN",
                        _dims="M*L^-3", _units="kg/m³", 
                        relevant=True, _schema=schema),
    "v": Variable(_name="Velocity", _sym="v", _cat="OUT",
                    _dims="L*T^-1", _units="m/s",
                    relevant=True, _schema=schema),
    "D": Variable(_name="Diameter", _sym="D", _cat="IN",
                    _dims="L", _units="m",
                    relevant=True, _schema=schema),
    "\\mu": Variable(_name="Viscosity", _sym="\\mu", _cat="IN",
                    _dims="M*L^-1*T^-1", _units="Pa·s",
                    relevant=True, _schema=schema),
    "g": Variable(_name="Gravity", _sym="g", _cat="CTRL",
                    _dims="L*T^-2", _units="m/s²",
                    relevant=False, _schema=schema)  # Irrelevant for Reynolds
}

# Create analysis engine and run workflow
engine = AnalysisEngine(_name="Reynolds Number Analysis",
                        _fwk="CUSTOM",
                        _schema=schema,
                        _variables=variables)

# Execute dimensional analysis workflow
engine.create_matrix()  # Constructs dimensional matrix
coefficients = engine.solve()  # Generates π coefficients

# Access Reynolds number coefficient
reynolds = coefficients["\\Pi_{0}"]
print(f"Reynolds Formula: {reynolds.pi_expr}")
print(f"Variables: {list(reynolds.variables.keys())}")

# Export complete analysis to dictionary (now properly serializes nested objects)
analysis_data = engine.to_dict()

# Create data directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Save to JSON file for integration with other systems
with open("data/reynolds_da.json", "w") as f:
    json.dump(analysis_data, f, indent=4)

print("Analysis exported to data/reynolds_da.json")

# Later: Reload analysis from JSON
with open("data/reynolds_da.json", "r") as f:
    loaded_data = json.load(f)

# Reconstruct engine from dictionary
engine_restored = AnalysisEngine.from_dict(loaded_data)
print(f"Restored coefficients: {list(engine_restored.coefficients.keys())}")

Reynolds Formula: \frac{\rho*D*v}{\mu}
Variables: ['\\rho', 'D', '\\mu', 'v']
Analysis exported to data/reynolds_da.json
Restored coefficients: ['\\Pi_{0}']


In [25]:
# Sensitivity Analysis Example
from pydasa import Variable, Schema, AnalysisEngine
from pydasa.dimensional.fundamental import Dimension
from pydasa.workflows.influence import SensitivityAnalysis

# ========================================================================
# STEP 1: Run Dimensional Analysis (Precondition)
# ========================================================================

# Define custom framework (T, M, L only)
custom_fdus = [
    Dimension(_idx=0, _sym="T", _unit="s", _name="Time"),
    Dimension(_idx=1, _sym="M", _unit="kg", _name="Mass"),
    Dimension(_idx=2, _sym="L", _unit="m", _name="Length")
]
schema = Schema(_fwk="CUSTOM", _fdu_lt=custom_fdus)

# Define variables with numerical bounds for sensitivity analysis
variables = {
    "\\rho": Variable(_name="Density",
                      _sym="\\rho",
                      _cat="IN",
                      _dims="M*L^-3",
                      _units="kg/m³",
                      _std_mean=1000.0,
                      _std_min=990.0,
                      _std_max=1020.0,
                      relevant=True,
                      _schema=schema),
    "v": Variable(_name="Velocity",
                  _sym="v",
                  _cat="OUT",
                  _dims="L*T^-1",
                  _units="m/s",
                  _std_mean=5.0,
                  _std_min=1.0,
                  _std_max=6.0,
                  relevant=True,
                  _schema=schema),
    "D": Variable(_name="Diameter",
                  _sym="D",
                  _cat="IN",
                  _dims="L",
                  _units="m",
                  _std_mean=0.05,
                  _std_min=0.04,
                  _std_max=0.06,
                  relevant=True,
                  _schema=schema),
    "\\mu": Variable(_name="Viscosity",
                     _sym="\\mu",
                     _cat="IN",
                     _dims="M*L^-1*T^-1",
                     _units="Pa·s",
                     _std_mean=0.001002,
                     _std_min=0.0009,
                     _std_max=0.0011,
                     relevant=True,
                     _schema=schema),
    "g": Variable(_name="Gravity",
                  _sym="g",
                  _cat="CTRL",
                  _dims="L*T^-2",
                  _units="m/s²",
                  _std_mean=9.81,
                  _std_min=9.80,
                  _std_max=9.82,
                  relevant=False,
                  _schema=schema)  # Excluded from matrix
}

# Create and run dimensional analysis
engine = AnalysisEngine(_name="Reynolds Number Analysis",
                        _fwk="CUSTOM",
                        _schema=schema,
                        _variables=variables)

engine.create_matrix()
coefficients = engine.solve()

print(f"Dimensional analysis complete: {list(coefficients.keys())}")

# ========================================================================
# STEP 2: Perform Sensitivity Analysis
# ========================================================================

# Create sensitivity analysis workflow
sensitivity = SensitivityAnalysis(_name="Reynolds Sensitivity",
                                   _fwk="CUSTOM",
                                   _cat="SYM",
                                   _schema=schema,
                                   _variables=variables,
                                   _coefficients=coefficients)

# Run symbolic sensitivity analysis at mean values
results = sensitivity.analyze_symbolic(val_type="mean")

# Display results for Reynolds number coefficient
re_key = "SEN_{\\Pi_{0}}"
print("\n=== Symbolic Sensitivity Analysis Results ===")
if re_key in results:
    print(f"\nSensitivity for {re_key}:")
    for var_sym, sensitivity_val in results[re_key].items():
        if var_sym in variables:
            var_name = variables[var_sym].name
        else:
            var_name = var_sym
        print(f"\t∂π/∂{var_sym} ({var_name}): {sensitivity_val:+.4e}")

# ========================================================================
# STEP 3: Numerical Sensitivity (Alternative)
# ========================================================================

# Switch to numerical analysis mode
sensitivity.cat = "NUM"

# Run FAST (Fourier Amplitude Sensitivity Test)
numerical_results = sensitivity.analyze_numeric(n_samples=1000)

print("\n=== Numerical Sensitivity Analysis Results (FAST) ===")
for coeff_key, sens_data in numerical_results.items():
    print(f"\nSensitivity for {coeff_key}:")
    if "S1" in sens_data:
        # First-order sensitivity indices
        s1_indices = sens_data["S1"]
        for i, var_sym in enumerate(sensitivity.variables.keys()):
            if var_sym in variables:
                var_name = variables[var_sym].name
            else:
                var_name = var_sym
            
            # S1 is a list, access by index
            # S1 represent the contribution of each input variable to the output variance
            if i < len(s1_indices):
                s1_val = s1_indices[i]
                print(f"\tS1: [{var_sym}] ({var_name}): {s1_val:.4f}")

Dimensional analysis complete: ['\\Pi_{0}']

=== Symbolic Sensitivity Analysis Results ===

Sensitivity for SEN_{\Pi_{0}}:
	∂π/∂D (Diameter): +4.9900e+06
	∂π/∂\mu (Viscosity): -2.4900e+08
	∂π/∂\rho (Density): +2.4950e+02
	∂π/∂v (Velocity): +4.9900e+04

=== Numerical Sensitivity Analysis Results (FAST) ===

Sensitivity for SEN_{\Pi_{0}}:
	S1: [\rho] (Density): 0.0004
	S1: [v] (Velocity): 0.0708
	S1: [D] (Diameter): 0.8909
	S1: [\mu] (Viscosity): 0.0175


In [ ]:
from pydasa import Variable, Schema, AnalysisEngine
from pydasa.dimensional.fundamental import Dimension
from pydasa.workflows.practical import MonteCarloSimulation
import random
import numpy as np

# ========================================================================
# STEP 1: Run Dimensional Analysis (Precondition)
# ========================================================================

# Define custom framework (T, M, L only)
custom_fdus = [
    Dimension(_idx=0, _sym="T", _unit="s", _name="Time"),
    Dimension(_idx=1, _sym="M", _unit="kg", _name="Mass"),
    Dimension(_idx=2, _sym="L", _unit="m", _name="Length")
]
schema = Schema(_fwk="CUSTOM", _fdu_lt=custom_fdus)

# Define variables with distribution specifications
variables = {
    "\\rho": Variable(_name="Density",
                        _sym="\\rho",
                        _cat="IN",
                        _dims="M*L^-3",
                        _units="kg/m³",
                        _dist_type="uniform",
                        _dist_params={"a": 990.0, "b": 1020.0},
                        _dist_func=lambda: random.uniform(990.0, 1020.0),
                        relevant=True,
                        _schema=schema),
    "v": Variable(_name="Velocity",
                    _sym="v",
                    _cat="OUT",
                    _dims="L*T^-1",
                    _units="m/s",
                    _dist_type="uniform",
                    _dist_params={"a": 1.0, "b": 6.0},
                    _dist_func=lambda: random.uniform(1.0, 6.0),
                    relevant=True,
                    _schema=schema),
    "D": Variable(_name="Diameter",
                    _sym="D",
                    _cat="IN",
                    _dims="L",
                    _units="m",
                    _dist_type="uniform",
                    _dist_params={"a": 0.04, "b": 0.06},
                    _dist_func=lambda: random.uniform(0.04, 0.06),
                    relevant=True,
                    _schema=schema),
    "\\mu": Variable(_name="Viscosity",
                        _sym="\\mu",
                        _cat="IN",
                        _dims="M*L^-1*T^-1",
                        _units="Pa·s",
                        _dist_type="constant",
                        _dist_params={"value": 0.001002},
                        _dist_func=lambda: 0.001002,
                        relevant=True,
                        _schema=schema),
    "g": Variable(_name="Gravity",
                    _sym="g",
                    _cat="CTRL",
                    _dims="L*T^-2",
                    _units="m/s²",
                    _dist_type="constant",
                    _dist_params={"value": 9.81},
                    _dist_func=lambda: 9.81,
                    relevant=False,
                    _schema=schema)  # Excluded from matrix
}

# Create and run dimensional analysis
engine = AnalysisEngine(_name="Reynolds Number Analysis",
                        _fwk="CUSTOM",
                        _schema=schema,
                        _variables=variables)

engine.create_matrix()
coefficients = engine.solve()

print(f"Dimensional analysis complete: {list(coefficients.keys())}")

# ========================================================================
# STEP 2: Perform Monte Carlo Simulation
# ========================================================================

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Create Monte Carlo simulation workflow
mc_simulation = MonteCarloSimulation(_name="Reynolds Monte Carlo",
                                        _fwk="CUSTOM",
                                        _cat="DIST",
                                        _experiments=1000,
                                        _variables=variables,
                                        _coefficients=coefficients)

# Run simulation
mc_simulation.run_simulation(iters=1000)

print(f"✓ Monte Carlo simulation complete")
print(f"  Experiments: {mc_simulation.experiments}")
print(f"  Simulations: {list(mc_simulation.simulations.keys())}")

# ========================================================================
# STEP 3: Extract Results and Statistics
# ========================================================================

# Get Reynolds number simulation results
re_key = "\\Pi_{0}"
re_simulation = mc_simulation.get_simulation(re_key)

print(f"\nReynolds Number Statistics:")
print(f"\tMean: {re_simulation.mean:.2e}")
print(f"\tMedian: {re_simulation.median:.2e}")
print(f"\tStd Dev: {re_simulation.dev:.2e}")
print(f"\tMin: {re_simulation.min:.2e}")
print(f"\tMax: {re_simulation.max:.2e}")

# Extract result arrays for further analysis
results_dict = mc_simulation.results
re_results = results_dict[re_key]["results"]

print(f"\nResult array shape: {re_results.shape}")
print(f"\tFirst 5 results: {re_results[:5]}...")

Dimensional analysis complete: ['\\Pi_{0}']
✓ Monte Carlo simulation complete
  Experiments: 1000
  Simulations: ['\\Pi_{0}']

Reynolds Number Statistics:
  Mean: 1.75e+05
  Median: 1.73e+05
  Std Dev: 7.50e+04
  Min: 4.58e+04
  Max: 3.54e+05

Result array shape: (1000,)
First 5 results: [ 51557.56468247 249338.40800126  70518.40469616 103721.16447116
 104505.61895793]...
